Introduction to xAct
=============

Jiwon Park, November 29, 2022

What is xAct?
-------------
xACt is a Mathematica package to conduct symbolic computation in general relativity. It has the broadest usage among codes currently out, allowing many techniques including curvature computation, coordinates transformation, rule-based computation, action principle and perturbation theory.

load package
-------------

In [ ]:
ClearAll["Global`*"];
Quiet@Block[{Print},
  << xAct`xTensor`; << xAct`xCoba`;] 
(*quiet and block command are just used to suppress messy output*)
removeAll := Remove[Evaluate[$Context <> "*"]];
(*
To clear symbols so that no error arise when running codes.
Credit to: https://mathematica.stackexchange.com/questions/850/how-do-i-clear-\all-user-defined-symbols
*)

Don't be alarmed if a new DOS window opens! It's just an external computation module supporting xAct.

declare manifold and tensors
-------------
M denotes manifold, 4 is dimension, a,b,c,d are coordinates indices.

In [ ]:
DefManifold[M, 4, {a, b, c, d}]

In [ ]:
DefScalarFunction /@ {X, Y};
DefScalarFunction /@ {R, H}; (*scale factor and Hubble factor*)
R'[t_] := H[t] R[t]
R''[t_] := D[H[t] R[t], t]

In [ ]:
DefTensor[v[a], M] (*covariant index*)
v[a]//TeXForm

In [ ]:
DefTensor[w[-a], M] (*contravariant index*)
w[-a]//TeXForm

we can define (anti-)symmetric tensors easily.

In [ ]:
DefTensor[F[-a, -b], M, Symmetric[{-a, -b}]]
F[-a, -b]//TeXForm

In [ ]:
F[-b, -a] // ToCanonical//TeXForm

In [ ]:
DefTensor[G[-a, -b], M, Antisymmetric[{-a, -b}]]
G[-a, -b]//TeXForm

In [ ]:
G[-b, -a] // ToCanonical//TeXForm

The Mathematica's standard function 'Simplify' does not carry out full tensor manipulation. Instead, we use 'ToCanonical.'

In [ ]:
v[a] v[b] G[-a, -b] // Simplify//TeXForm

In [ ]:
v[a] v[b] G[-a, -b] // ToCanonical//TeXForm

In [ ]:
(v[a] v[b] F[-a, -b] + v[a] v[c] F[-a, -c]) // Simplify//TeXForm

In [ ]:
(v[a] v[b] F[-a, -b] + v[a] v[c] F[-a, -c]) // ToCanonical//TeXForm

chart and coordinates
----------
To convey computations under a specific coordinates, we use 'DefChart' function.

In [ ]:
Quiet@Block[
  {Print},
    DefChart[Cartesian, M, {0, 1, 2, 3}, {t[], x[], y[], z[]}, 
      ChartColor -> Green];
    DefChart[Spherical, M, {0, 1, 2, 3}, {t[], r[], \[Theta][], \[Phi][]},
      ChartColor -> Red];
]

Under the given chart, we can now assign matrix components for tensors.

In [ ]:
matrixComponent =
  {
   {X[t[]], Y[t[]], Y[t[]], Y[t[]]},
   {Y[t[]], X[t[]], 0, 0},
   {Y[t[]], 0, X[t[]], 0},
   {Y[t[]], 0, 0, X[t[]]}
   };
matrixComponent // MatrixForm//TeXForm

In [ ]:
F[-a, -b] // ComponentArray//TeXForm
(*component array before chart declaration*)

In [ ]:
F[{-a, -Cartesian}, {-b, -Cartesian}] // ComponentArray//TeXForm
(*component array under Cartesian coordinates*)

we use 'ComponentValue' to assign values.

In [ ]:
ComponentValue[F[-{a,Cartesian}, -{b, Cartesian}]// ComponentArray, matrixComponent] //TeXForm

In [ ]:
ComponentValue[ G[{-a, -Cartesian}, {-b, -Cartesian}]//ComponentArray, matrixComponent]//TeXForm

In [ ]:
F[-a, -b]//TeXForm

We cannot show values without a specific coordinates. 

In [ ]:
F[-1, -1]//TeXForm

To set coordinates, we use 'ToBasis' function.

In [ ]:
F[-a, -b] // ToBasis[Cartesian]//TeXForm

In [ ]:
F[-a, -b] // ToBasis[Cartesian] // ComponentArray//TeXForm

In [ ]:
F[-a, -b] // ToBasis[Cartesian] // ComponentArray // ToValues //TeXForm

In [ ]:
F[-a, -b] // ToBasis[Cartesian] // ComponentArray // ToValues //TeXForm
%[[1, 1]]//TeXForm
(*note that Mathematica matrix indices begin with 1, not 0!*)

Let us put things together!

※ For those who wonder why we apply 'ToValues' function many times:
in fact it depends on the situation. Since xAct package does not support auto-evaluation function like 'Evaluate' in Mathematica, we need to iterate 'ToValues' to fully evaluate every component of a tensor. For most cases, it suffices to apply twice or triple.
You may modify 'MyArrayComponents' to see how the output varies by changing the number of iterations.

In [ ]:
MyArrayComponents[expr_] := 
 expr // ToBasis[Cartesian] // ComponentArray // ToValues // 
     ToValues // ToValues // Simplify // MatrixForm

In [ ]:
F[-a, -b] // MyArrayComponents//TeXForm

Note that matrix components are automatically changed to follow previously declared rules.

In [ ]:
G[-a, -b] // MyArrayComponents//TeXForm

In [ ]:
F[a, -b] // MyArrayComponents

In [ ]:
metricMatrix =
  {
   {-1, 0, 0, 0},
   {0, Z[t[]]^2, 0, 0},
   {0, 0, Z[t[]]^2, 0},
   {0, 0, 0, Z[t[]]^2}
   };
metricMatrix // MatrixForm//TeXForm

metric, covariant derivatives, curvature
-------
Let us define metric tensor under Cartesian coordinates.

In [ ]:
DefMetric[-1, g[-a, -b], covd, {";", "\[Del]"}, PrintAs -> "g"];
MatrixForm@MetricInBasis[g, -Cartesian, metricMatrix]

In [ ]:
MetricCompute[g, Cartesian, All]

In [ ]:
g[{a, Cartesian}, {b, Cartesian}] // MyArrayComponents//TeXForm

Tensor indices cannot be automatically raised or lowered. To make this operations possible, we have to use 'ChangeComponents' function.
ChangeComponents[B,A] defines B from A, metric and rules.

In [ ]:
ChangeComponents[F[{a, Cartesian}, -{b, Cartesian}], 
  F[-{a, Cartesian}, -{b, Cartesian}]]; 
ChangeComponents[F[-{a, Cartesian}, {b, Cartesian}], 
  F[-{a, Cartesian}, -{b, Cartesian}]]; 

In [ ]:
F[-a, b] // MyArrayComponents//TeXForm

Note that metric compatibility is naturally satisfied because $\nabla$ is defined from the metric.

In [ ]:
covd[c]@g[a, b]//TeXForm

We can also obtain curvature tensors directly.

In [ ]:
Riccicovd[-a, -b] // MyArrayComponents
Einsteincovd[-a, -b] // MyArrayComponents

Rule-based computation
--------
However, we cannot evaluate derivatives in general. 

In [ ]:
matrixComponent2 =
  {
   {X[t[]], 0, 0, 0},
   {0, X[t[]], 0, 0},
   {0, 0, X[t[]], 0},
   {0, 0, 0, X[t[]]}
   };
matrixComponent2 // MatrixForm
ComponentValue[ F[-{a, Cartesian}, -{b, Cartesian}] // ComponentArray, matrixComponent2]

In [ ]:
covd[a]@F[-a, -b]//TeXForm

But we may compute Christoffel symbols 

In [ ]:
ChristoffelcovdPDCartesian[a, -b, -c]
% // MyArrayComponents

Although the evaluation with 'CovDToChristoffel' does not give an useful value, it shows how we should assign values.

In [ ]:
covd[a]@F[-a, -b] // CovDToChristoffel
(*ignore random number behind dummy indices. they are just internal pointing values.*)

In [ ]:
EvaluationOfCovD[covd[a], F[-a, b_]] := Module[{index = b},
   coord[a_] := {t[], x[], y[], z[]}[[a + 1]];
   MyChristoffel = 
    ChristoffelcovdPDCartesian[a, -c, -d] // ToBasis[Cartesian] // 
         ComponentArray // ToValues // ToValues // ToValues // 
     Simplify;
   (*we do not change it to MatrixForm. 
   otherwise it will cause unnessasary error.*)
   result =
    (-Sum[
          MyChristoffel[[d + 1, a + 1, index + 1]] ToValues@
            F[{a, Cartesian}, {d, -Cartesian}]
          , {a, 0, 3}, {d, 0, 3}] // ToValues // Simplify)
     + (-Sum[
          g[{a, Cartesian}, {c, Cartesian}]
           MyChristoffel[[d + 1, c + 1, a + 1]] ToValues@
            F[{d, -Cartesian}, {index, -Cartesian}]
          , {a, 0, 3}, {c, 0, 3}, {d, 0, 3}] // ToValues // Simplify)
     + (Sum[
         g[{a, Cartesian}, {c, Cartesian}]
          D[ToValues@F[{a, -Cartesian}, {index, -Cartesian}], coord[c]]
         , {a, 0, 3}, {c, 0, 3}] // ToValues // Simplify);
   result // Simplify
   ];

In [ ]:
EvaluationOfCovD[covd[a], F[-a, 0]]//TeXForm

coordinates change
--------
We can define a relation between two coordinates. Note that this may be exploited in tetrad formalism.

In [ ]:
Basis[{a, Cartesian}, {-b, -Spherical}]

In [ ]:
x /: InChart[x[], Spherical] = 
  Function[{t, r, \[Theta], \[Phi]}, r Cos[\[Phi]] Sin[\[Theta]]];
y /: InChart[y[], Spherical] = 
  Function[{t, r, \[Theta], \[Phi]}, r Sin[\[Phi]] Sin[\[Theta]]];
z /: InChart[z[], Spherical] = 
  Function[{t, r, \[Theta], \[Phi]}, r Cos[\[Theta]]];
r /: InChart[r[], Cartesian] = 
  Function[{t, x, y, z}, Sqrt[x^2 + y^2 + z^2]];
\[Theta] /: InChart[\[Theta][], Cartesian] = 
  Function[{t, x, y, z}, ArcCos[z/Sqrt[x^2 + y^2 + z^2]]];
\[Phi] /: InChart[\[Phi][], Cartesian] = 
  Function[{t, x, y, z}, ArcTan[x, y]];
$Assumptions = And[
   {t[], x[], y[], z[], r[], \[Phi][], \[Theta][]} \[Element] Reals,
   r[] >= 0,
   0 <= \[Theta][] <= Pi,
   0 <= \[Phi][] < 2 Pi
   ];

Let us check our function works well by conducting transformation and its inverse.

In [ ]:
ScalarsOfChart[Cartesian]
InChart[Spherical] /@ %
(InChart[Cartesian] /@ %) // Simplify

We need an additional package to apply this to general tensors. 

In [ ]:
Quiet@Block[{Print}, << xAct`xTras`]
MatrixForm /@ ComputeBasisValues[Cartesian, Spherical]

In [ ]:
F[-a, -b] // ToBasis[Spherical] // ComponentArray

In [ ]:
F[-a, -b] // ToBasis[Spherical] // SeparateBasis[Cartesian] // ComponentArray

In [ ]:
F[-a, -b] // ToBasis[Spherical] // SeparateBasis[Cartesian] // ComponentArray // TraceBasisDummy // MyArrayComponents

CTensor: more convenient way to compute tensor components
--------

In [ ]:
removeAll
removeAll := Remove[Evaluate[$Context <> "*"]];

In [ ]:
Quiet@Block[{Print}, << xAct`xCoba`]
DefManifold[M, 4, {a, b, c, d}];
DefScalarFunction /@ {X, R, H}; (*scale factor and Hubble factor*)
R'[t_] := H[t] R[t]
R''[t_] := D[H[t] R[t], t]
metricMatrix =
  {
   {-1, 0, 0, 0},
   {0, Z[t[]]^2, 0, 0},
   {0, 0, Z[t[]]^2, 0},
   {0, 0, 0, Z[t[]]^2}
   };
matrixComponent =
  {
   {X[t[]], 0, 0, 0},
   {0, X[t[]], 0, 0},
   {0, 0, X[t[]], 0},
   {0, 0, 0, X[t[]]}
   };
Quiet@Block[{Print}, DefChart[Cartesian, M, {0, 1, 2, 3}, {t[], x[], y[], z[]}, ChartColor -> Green]];

In [ ]:
g = CTensor[metricMatrix, {-Cartesian, -Cartesian}];

In [ ]:
g[-a, -b]

In [ ]:
SetCMetric[g, Cartesian, SignatureOfMetric -> {3, 1, 0}]

In [ ]:
g[a, b]

In [ ]:
F = CTensor[matrixComponent, {-Cartesian, -Cartesian}];
F[-a, b]

In [ ]:
CD = LC[g];

In [ ]:
CD[c]@g[a, b]

In [ ]:
CD[a]@F[-a, -b]

To show long complex components, we adjust display parameter.

In [ ]:
xAct`xCoba`$LargeComponentSize = 3000;
CD[a]@F[-a, -b]

In [ ]:
Einstein[CD][-a, -b]

action principle
--------

In [ ]:
removeAll
removeAll := Remove[Evaluate[$Context <> "*"]];

In [ ]:
ClearAll["Global`*"];
Quiet@Block[{Print}, << xAct`xPert`]
DefManifold[M, 4, {a, b, c, d}]

In [ ]:
PerturbAction[expr_, 
  g_?MetricQ[a_?UpIndexQ, b_?UpIndexQ] | 
   g_?MetricQ[a_?DownIndexQ, b_?DownIndexQ]] := 
 Module[{pertexpr, res, dgloc,(*dummyloc,*)hp},
  
  (* We define the metric perturbation, if not defined already *)
  dgloc = SymbolJoin["\[Delta]", g];
  
  hp = Head@Perturbation[g[DownIndex@a, DownIndex@b]];
  
  If[hp === Perturbation, 
   DefMetricPerturbation[g, dgloc, SymbolJoin["\[Epsilon]", g]], 
   dgloc = hp];
  
  
  Block[{$DefInfoQ = False},
   
   (* We perturb wrt to the metric and if it is the inverse metric we \
put a minus sign *)
   pertexpr = (If[DownIndexQ[a], 1, -1])*
      ToCanonical@
       ContractMetric@ExpandPerturbation@Perturbation[expr] /. 
     Perturbation[tens_] :> 0;
   
   (*We then use VarD. 
   It happens that some trivial Kronecker appear which need to be \
handle manually *)
   res = 
    ToCanonical[(SameDummies@
        ContractMetric@
         VarD[dgloc[LI[1], a, b], CovDOfMetric[g]][pertexpr]) /. 
      delta[-LI[n_], LI[m_]] :> 
       KroneckerDelta[NoScalar[n], NoScalar[m]]];
   ];
  res
  ]
PerturbAction[expr_, tensor_?xTensorQ, covd_] := 
 Module[{res, dummyloc, pertexpr, inds},
  Block[{$DefInfoQ = False},
   
   (* We use a dummy name for the variation of the tensor, 
   and use it to replace the formal first order perturbation the \
tensor *)
   (* So first we define this dummy tensor *)
   dummyloc = SymbolJoin["Var", tensor];
   inds = DummyIn /@ SlotsOfTensor[tensor];
   If[! xTensorQ[dummyloc], 
    DefTensor[dummyloc @@ inds, First@DependenciesOfTensor@tensor]];
   SymmetryGroupOfTensor[dummyloc] ^= SymmetryGroupOfTensor[tensor];
   
   (* Then we perturb the action and replace Perturbation[
   Tensor[..]] by this dummy tensor *)
   pertexpr = (ToCanonical@
      ContractMetric[
       ExpandPerturbation@Perturbation[expr] /. 
         Perturbation[tens_?((# =!= tensor) &)[ar___]] :> 0 /. 
        Perturbation[tensor[ind___]] :> dummyloc[ind]]);
   
   (* With this simple head, VarD works correctly. 
   Again we need to handel some trivial Kronecker *)
   res = 
    ToCanonical[(SameDummies@
        ContractMetric@VarD[dummyloc @@ inds, covd][pertexpr]) /. 
      delta[-LI[n_], LI[m_]] :> 
       KroneckerDelta[NoScalar[n], NoScalar[m]]];
   ];
  res
  ]
PerturbAction[expr_, tensor_[inds___]] := 
 PerturbAction[expr, tensor[inds], CovDOfMetric@First@$Metrics]
PerturbAction[expr_, tensor_?xTensorQ[inds___], covd_] := 
 Module[{res, dummyloc, pertexpr},
  Block[{$DefInfoQ = False},
   dummyloc = SymbolJoin["Var", tensor];
   
   If[! xTensorQ[dummyloc], 
    DefTensor[dummyloc[inds], First@DependenciesOfTensor@tensor]];
   SymmetryGroupOfTensor[dummyloc] ^= SymmetryGroupOfTensor[tensor];
   
   (* Perturbation with xPert*)
   pertexpr = (ToCanonical@
      ContractMetric[
       ExpandPerturbation@Perturbation[expr] /. 
         Perturbation[tens_?((# =!= tensor) &)[ar___]] :> 0 /. 
        Perturbation[tensor[ind___]] :> dummyloc[ind]]);
   (* VarD and removal of KroneckerDelta*)
   res = 
    ToCanonical[(SameDummies@
        ContractMetric@VarD[dummyloc[inds], covd][pertexpr]) /. 
      delta[-LI[n_], LI[m_]] :> 
       KroneckerDelta[NoScalar[n], NoScalar[m]]];
   ];
  res
  ]

In [ ]:
VarAction[expr_, 
  g_?MetricQ[as__?((UpIndexQ[#] || DownIndexQ[#]) &)]] := 
 Module[{sqrtg},
  sqrtg = Sqrt[SignDetOfMetric[g] Determinant[g][]];
  ToCanonical[
   PerturbAction[expr, g[as]] + 
    ReplaceDummies@expr*PerturbAction[sqrtg, g[as]]/sqrtg ]
  ]
VarAction[expr_, tensor_?((xTensorQ[#] && Not[MetricQ[#]]) &), 
  g_?MetricQ] := PerturbAction[expr, tensor, CovDOfMetric[g]]
VarAction[expr_, tensor_?((xTensorQ[#] && Not[MetricQ[#]]) &)] := 
  VarAction[expr, tensor, First@$Metrics];

VarAction[expr_, 
  tensor_?((xTensorQ[#] && Not[MetricQ[#]]) &)[inds___], g_?MetricQ] :=
  PerturbAction[expr, tensor[inds], CovDOfMetric[g]]
VarAction[expr_, 
   tensor_?((xTensorQ[#] && Not[MetricQ[#]]) &)[inds___]] := 
  VarAction[expr, tensor[inds], First@$Metrics];

In [ ]:
Quiet@Block[{Print},
  DefTensor[\[Phi][], M];
  DefScalarFunction[V];
  
  DefTensor[A[-a], M];
  DefTensor[F[-a, -b], M, Antisymmetric[{-a, -b}]] ;
  IndexSetDelayed[F[a_, b_], CD[a][A[b]] - CD[b][A[a]]];
  
  DefMetric[-1, g[-a, -b], CD, {";", "\[Del]"}, PrintAs -> "g"];
  ]

Scalar field Lagrangian

In [ ]:
L = 1/2 CD[a]@\[Phi][] CD[-a]@\[Phi][] - V[\[Phi][]];
VarAction[L, g[a, b]];
%//TeXForm
VarAction[L, \[Phi][]];
%//TeXForm

Maxwell Lagrangian

In [ ]:
L = F[a, b] F[-a, -b]/4;
VarAction[L, g[a, b]];
%//TeXForm
VarAction[L, A[a]];
%//TeXForm

Hilbert - Einstein Lagrangian

In [ ]:
L = RicciScalarCD[];
VarAction[L, g[a, b]];
%//TeXForm

Neat example: $ R + \alpha R^2 + \beta R_{ab} R^{ab} $ gravity
--------

First some definitions to construct Lagrangian:

In [ ]:
ClearAll[\[Alpha], \[Beta]];
DefTensor[\[Alpha][], M];
DefTensor[\[Beta][], M];
L = RicciScalarCD[] + \[Alpha][] (RicciScalarCD[])^2 + \[Beta][] \
RicciCD[-a, -b] RicciCD[a, b]

Then we vary the action with the metric tensor.

In [ ]:
C1Rule = MakeRule[{CD[a]@\[Alpha][], 0}];
C2Rule = MakeRule[{CD[a]@\[Beta][], 0}];
(*To assure C1 and C2 are coefficients*)
VariedAction = 
 VarAction[L, g[a, b]] /. C1Rule /. C2Rule // FullSimplify

It is impossible to evaluate the above result using CTensor, so we make a temporary file to store the result.

In [ ]:
Export[ToString[NotebookDirectory[]] <> "VariedAction.png", Rasterize[NotebookRead[PreviousCell[]], ImageResolution -> 100, ImageSize -> Large]];

To find Friedmann equations, we have to redefine Chart, metric, etc. again.

In [ ]:
removeAll;
removeAll := Remove[Evaluate[$Context <> "*"]];
Quiet@Block[{Print},
  << xAct`xCoba` << xAct`xTras`;
  DefManifold[M, 4, {a, b, c, d}];
  DefChart[Cartesian, M, {0, 1, 2, 3}, {t[], x[], y[], z[]}];
  DefScalarFunction /@ {R, H}; 
  DefScalarFunction /@ {\[Alpha], \[Beta]}; 
  R'[t_] := H[t] R[t];
  R''[t_] := D[H[t] R[t], t];
  metricMatrix =
   {
    {-R[t[]]^2, 0, 0, 0},
    {0, R[t[]]^2, 0, 0},
    {0, 0, R[t[]]^2, 0},
    {0, 0, 0, R[t[]]^2}
    };
  g = CTensor[metricMatrix, {-Cartesian, -Cartesian}];
  SetCMetric[g, Cartesian, SignatureOfMetric -> {3, 1, 0}];
  covd = LC[g];
  MetricCompute[g, Cartesian, All];]

In [ ]:
Import[ToString[NotebookDirectory[]] <> "VariedAction.png"]

We need to convert the above output to the corresponding CTensor form manually... (a bit tedious but no other way!)

In [ ]:
MyArrayComponents[expr_] := 
 expr // ToBasis[Cartesian] // ComponentArray // ToValues // 
    ToValues // ToValues // Simplify
Eq = Ricci[covd][-a, b] (1 + \[Alpha][] RicciScalar[covd][[1]]) - 
    2 \[Alpha][] covd[b][
      covd[-a][
       RicciScalar[covd][[1]]]] + \[Beta][] (2 Ricci[covd][-a, 
         c] Ricci[covd][b, -c] - 
       covd[-c][covd[-a][Ricci[covd][b, c]]] - 
       covd[-c][covd[b][Ricci[covd][-a, c]]] + 
       covd[-c][covd[c][Ricci[covd][-a, b]]]) - 
    1/2 g[-a, 
      b] (RicciScalar[
         covd][[1]] + \[Alpha][] RicciScalar[covd][[1]]^2 + 
       Ricci[covd][-c, -d] Ricci[covd][c, d] - 
       4 \[Alpha][] covd[-c][covd[c][RicciScalar[covd][[1]]]] - 
       2 \[Beta][] covd[-d][covd[-c][Ricci[covd][c, d]]]) // 
   MyArrayComponents;

Let' s now define the energy-momentum tensor. We first define an observer' s 4-velocity vector $u^{\mu}$.

In [ ]:
DefTensor[u[a], M];
uComponents = {Simplify[
    1/Sqrt[-g[{0, -Cartesian}, {0, -Cartesian}]] // ToValues, 
    Assumptions -> R[t[]] > 0], 0, 0, 0};
u = CTensor[uComponents, {Cartesian}];

and then components of the energy-momentum tensor.

In [ ]:
DefTensor[T[-a, -b], M, Symmetric[{-a, -b}]];
DefScalarFunction[P];
DefScalarFunction[\[Rho]];
IndexSet[
  T[a_, b_], \[Rho][t[]] u[a] u[b] + P[t[]] (g[a, b] + u[a] u[b])];

In [ ]:
EMTensor = T[-a, b] // MyArrayComponents;

Now let's find Friedmann equation for this theory!

In [ ]:
DefConstantSymbol[\[Kappa]];(* 8\[Pi]G*)
EinsteinEq[a_, b_] := Eq[[a, b]] == \[Kappa] EMTensor[[a, b]];

temporal and spatial terms. Note that here we did not use Bianchi identity to simplify equations so there appears higher derivative terms, i.e., $H^{3}[t]$.

In [ ]:
G00Eq = Collect[EinsteinEq[1, 1], {H[t[]], H'[t[]]}, Simplify]//TeXForm
GrrEq = Collect[EinsteinEq[2, 2], {H[t[]], H'[t[]]}, Simplify]//TeXForm

A specific form to represent the equation on $H^\prime$:

In [ ]:
Collect[3 GrrEq - G00Eq, {H[t[]], H'[t[]]}, Simplify]//TeXForm

For matter conservation equation:

In [ ]:
Cons[b_] := Module[{a}, covd[-a]@T[a, b] == 0]
(Cons[b] // MyArrayComponents)[[1]]//TeXForm